In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
def readin_large_data():
    """Read small data set into a 2D numpy array
    return: numpy array contains the small data set, shape: (50,000, 230)
    """
    PATH = os.getcwd()
    
    DATA_PATH = os.path.join(PATH, 'data_large','orange_large_train.data','orange_large_train.data.chunk' + str(i))
    
    if i == 1:
        data = pd.read_csv(DATA_PATH, delimiter = '\t')
    else:
        data = pd.read_csv(DATA_PATH, delimiter='\t', header=None)
    data_type = {key:np.float64 for key in header[:190]}
    data_type.update({key:str for key in header[190:]})
    return data

large_data = readin_large_data(1)
print(large_data.shape)

In [2]:
PATH = os.getcwd()
DATA = os.path.join(PATH, 'data_large', 'orange_large_train.data','orange_large_train.data.chunk1')
with open(DATA) as f:
	header = f.readline().strip('\n').split('\t')
data_type = {key:np.float64 for key in header[:14740]}
data_type.update({key:str for key in header[14740:]})
large_data = pd.read_table(DATA, dtype=data_type)

for i in range(2, 6):
	DATA = os.path.join(PATH,'data_large', 'orange_large_train.data','orange_large_train.data.chunk' + str(i))
	temp = pd.read_table(DATA, header=None, dtype=data_type)
	temp.columns=header
	large_data = large_data.append(temp)

print('Pecent of missing value: ', large_data.isnull().sum().sum() / (large_data.shape[0] * large_data.shape[1]))

/afs/inf.ed.ac.uk/user/s17/s1719186/miniconda3/envs/dme/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14741,14742,14748,14749,14750,14752,14753,14755,14758,14759,14760,14762,14767,14769,14773,14776,14779,14780,14781,14789,14791,14797,14798,14800,14804,14805,14806,14809,14811,14817,14818,14819,14824,14828,14832,14834,14836,14843,14844,14850,14851,14855,14861,14871,14876,14878,14879,14881,14882,14883,14886,14888,14890,14894,14896,14897,14900,14904,14906,14914,14916,14917,14918,14926,14928,14931,14932,14936,14937,14939,14942,14944,14947,14951,14954,14957,14958,14970,14972,14980,14996,14998) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/afs/inf.ed.ac.uk/user/s17/s1719186/miniconda3/envs/dme/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14741,14742,14748,14749,14750,14752,14753,14755,14758,14759,14760,14762,1476

Pecent of missing value:  0.033478092


In [3]:
missing_rate = large_data.isnull().sum() / large_data.shape[0]

numerical_data = large_data.loc[:, large_data.columns[:-260]]
numerical_missing_rate = numerical_data.isnull().sum() / numerical_data.shape[0]
useful_numerical_data = numerical_data.loc[:, numerical_missing_rate < 0.5]

categorical_data = large_data.loc[:, large_data.columns[-260:]]
categorical_missing_data = categorical_data.isnull().sum() / categorical_data.shape[0]
useful_categorical_data = categorical_data.loc[:, categorical_missing_data < 0.5]

# useful_numerical_data.to_csv('numerical_data.csv')
# useful_categorical_data.to_csv('categorical_data.csv')
# sns.distplot(numerical_data.isnull().sum(axis=1) / data.shape[0], kde=False, rug=True, bins=10)
# t=numerical_data.isnull().sum(axis=1) / numerical_data.shape[1]
clean_numerical_data = useful_numerical_data.fillna(useful_numerical_data.mean())
clean_categorical_data = useful_categorical_data.fillna(useful_categorical_data.mode().iloc[0])

In [4]:
clean_numerical_data.as_matrix().shape

(50000, 14438)

In [5]:
clean_categorical_data.as_matrix().shape

(50000, 36)

In [6]:
clean_numerical_data.to_csv('large_clean_numerical_data.csv')
clean_categorical_data.to_csv('large_categorical_data.csv')

Read labels and run experiments.

In [ ]:
def readin_label(target):
    """Read upselling, churn or appetency label into a 2D numpy array
    parameter target: choose which label to load
    return: numpy array contains the specified label of small data set, shape: (50,000, 1)
    """
    assert target in ['upselling', 'churn', 'appetency']
    PATH = os.getcwd()
    LABEL_PATH = os.path.join(PATH, 'data_large','orange_large_train_'+target+'.labels')

    label = pd.read_csv(LABEL_PATH, delimiter='\t',prefix=target+'_label')
    
#     label = label[:10000,:]
    
    label[label==1] = True
    label[label==-1] = False
    
    return label

In [ ]:
print('Percent of upselling: ', np.sum(upselling_label) / len(upselling_label))
print('Percent of churn: ', np.sum(churn_label) / len(churn_label))
print('Percent of appetency: ', np.sum(appetency_label) / len(appetency_label))

print('Pecent of missing value: ', np.sum(data == None) / (data.shape[0] * data.shape[1]))
print('Pecent of missing value: ', data.isnull().sum().sum() / (data.shape[0] * data.shape[1]))

In [ ]:
upselling_label = readin_label('upselling')
print(upselling_label.shape)
churn_label = readin_label('churn')
print(churn_label.shape)
appetency_label = readin_label('appetency')
print(appetency_label.shape)
labels = np.hstack([appetency_label, churn_label, upselling_label])
labels = np.hstack([labels, ~np.any(labels, axis=1)[:,np.newaxis]])
labels = labels.astype(int)

labels_current = labels[:9999,:]

In [ ]:
import sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
import keras
from sklearn.cross_validation import train_test_split

model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=14438))
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
model.add(Dense(units=100, activation='relu'))
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
model.add(Dense(units=100, activation='relu'))
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
# model.add(Dense(units=100, activation='relu'))
# keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
# model.add(Dense(units=100, activation='relu'))
# keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

model.add(Dense(units=4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

# trainX, valX, trainY, valY = train_test_split(clean_numerical_data, labels, test_size=0.2, random_state=42)



model.fit(sklearn.preprocessing.robust_scale(clean_numerical_data), labels_current, epochs=100, batch_size=50, validation_split=0.2)